## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [109]:
import pandas as pd
import numpy as np
import random
import re

## 1. Предобработка данных

In [24]:
train = pd.read_csv(
    "../data/news_train.txt",
    encoding="utf-8",
    sep="\t",
    header=None,
    names=["topic", "header", "text"],
)
test = pd.read_csv(
    "../data/news_test.txt",
    encoding="utf-8",
    sep="\t",
    header=None,
    names=["topic", "header", "text"],
)


In [25]:
train

,topic,header,text
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...
...,...,...,...
14995,life,Составлен рейтинг лучших европейских пляжей 20...,Опубликован рейтинг лучших европейских пляжей ...
14996,media,В «Снобе» объяснили причину смены формата,Генеральный директор «Сноб медиа» Марина Гевор...
14997,economics,Минфин предложил штрафовать за биткоины на 50 ...,"Минфин разработал законопроект, устанавливающи..."
14998,life,Мэл Гибсон заплатит бывшей подруге 750 тысяч д...,Актер и режиссер Мэл Гибсон выплатит своей быв...


In [26]:
pd.concat([df.topic.value_counts() for df in [train, test]], axis=1)

,topic,topic
sport,2215,423
science,2156,466
media,2111,403
economics,2080,426
culture,2053,426
life,2033,415
forces,1225,245
business,554,90
travel,289,54
style,284,52


In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [6]:
word_re = re.compile(r"(?u)\b\w\w+\b")
train["sentences"] = [
    [morph.parse(word)[0].normal_form for word in word_re.findall(text.lower())]
    for text in train["text"]
]


In [12]:
test["sentences"] = [
    [morph.parse(word)[0].normal_form for word in word_re.findall(text.lower())]
    for text in test["text"]
]


In [11]:
train.to_csv("../data/news_train.csv", sep=';', index=False)
test.to_csv("../data/news_test.csv", sep=';', index=False)

In [7]:
train[['sentences']]

,sentences
0,"[нападать, вашингтон, кэпиталзти, александр, о..."
1,"[власть, мексика, объявить, подделка, статуя, ..."
2,"[южнокорейский, samsung, анонсировать, защитит..."
3,"[контрольный, дисциплинарный, комитет, кдк, рф..."
4,"[интернет, издание, hopes, fears, объявить, о,..."
...,...
14995,"[опубликовать, рейтинг, хороший, европейский, ..."
14996,"[генеральный, директор, сноб, медиа, марина, г..."
14997,"[минфин, разработать, законопроект, устанавлив..."
14998,"[актёр, и, режиссёр, мэл, гибсон, выплатить, с..."


## 2. Word2Vec

In [2]:
from gensim.models import Word2Vec

In [27]:
train = pd.read_csv("../data/news_train.csv", sep=';')
test = pd.read_csv("../data/news_test.csv", sep=';')
train['sentences'] = train['sentences'].apply(eval)
test['sentences'] = test['sentences'].apply(eval)

In [43]:
sentences = train['sentences'].values

In [76]:
w2v = Word2Vec(sentences=sentences, min_count=1, sg=1, workers=8)

In [85]:
w2v.train(sentences, total_examples=w2v.corpus_count, epochs=10)

(25762868, 29592730)

#### Семантические ассоциации:

In [86]:
w2v.wv.most_similar(positive=['россия'], topn=5)

[('рф', 0.8021003007888794),
 ('белоруссия', 0.7200120091438293),
 ('страна', 0.7039775848388672),
 ('российский', 0.6945366263389587),
 ('украина', 0.6726686358451843)]

In [101]:
w2v.wv.most_similar(positive=['спорт'], topn=5)

[('мутко', 0.7072204351425171),
 ('штрбск', 0.7050353288650513),
 ('мамедов', 0.7007424235343933),
 ('четырёхлетие', 0.6968165040016174),
 ('росохотрыболовсоюз', 0.6863040924072266)]

In [88]:
w2v.wv.most_similar(positive=["гейтс", "apple"], negative=["microsoft"], topn=5)

[('джобс', 0.5872300267219543),
 ('мелинда', 0.5586092472076416),
 ('баффет', 0.5486021637916565),
 ('гэбриел', 0.5448411703109741),
 ('milyoni', 0.5348674058914185)]

In [89]:
# гейтс - microsoft - билл + тим = кук
w2v.wv.most_similar(positive=["гейтс", "apple", "тим"], negative=["microsoft", "билл"], topn=2)

[('кук', 0.6005523204803467), ('джобс', 0.5835373401641846)]

In [90]:
# гейтс - microsoft - билл + стив = джобс
w2v.wv.most_similar(positive=["гейтс", "apple", "стив"], negative=["microsoft", "билл"], topn=2)

[('джобс', 0.6888144016265869), ('кук', 0.5103824138641357)]

In [92]:
# гейтс - сша + россия = предпрениматели
w2v.wv.most_similar(positive=["гейтс", "россия"], negative=["сша"], topn=3)

[('волож', 0.6117308139801025),
 ('гришин', 0.6058022975921631),
 ('михельсон', 0.6045231819152832)]

In [99]:
# путин - россия + турция = Реджеп Тайип Эрдоган
w2v.wv.most_similar(positive=["путин", "турция"], negative=["россия"], topn=3)

[('реджеп', 0.6605539321899414),
 ('тайип', 0.6324986815452576),
 ('тайипа', 0.6293382048606873)]

## 3. Классификация

In [125]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [104]:
X_train = [' '.join(sentence) for sentence in train['sentences']]
y_train = train['topic']
X_test = [' '.join(sentence) for sentence in test['sentences']]
y_test = test['topic']

In [106]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [199]:
uniform(loc=0, scale=1.5).rvs(size=5)

array([1.39381187, 1.30017646, 0.05179605, 0.17320993, 1.48430035])

In [200]:
log_reg = RandomizedSearchCV(
    LogisticRegression(max_iter=200),
    param_distributions={
        'C': uniform(loc=0, scale=4)
    },
    n_jobs=8,
)

naive_bayes = RandomizedSearchCV(
    MultinomialNB(),
    param_distributions={
        'alpha': uniform(loc=0, scale=1.5)
    },
    n_jobs=8,
)

linear_svc = RandomizedSearchCV(
    LinearSVC(),
    param_distributions={
        'C': uniform(loc=0, scale=4)
    },
    n_jobs=8,
)

In [122]:
linear_svc.estimator.__class__.__name__

'LinearSVC'

In [203]:
log_reg.fit(X_train_vec, y_train)

RandomizedSearchCV(estimator=LogisticRegression(max_iter=200), n_jobs=8,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000110A2257EE0>})

In [207]:
pd.DataFrame(log_reg.cv_results_)[["rank_test_score", "param_C", "mean_test_score", "std_test_score"]]



,rank_test_score,param_C,mean_test_score,std_test_score
0,1,3.981743,0.875200,0.002217
1,2,3.443572,0.874000,0.002076
2,2,3.23823,0.874000,0.002055
3,6,1.898327,0.869533,0.002050
4,4,2.829867,0.873133,0.001586
5,6,1.907186,0.869533,0.002050
6,5,2.42106,0.872133,0.001024
7,10,0.482826,0.844867,0.002754
8,8,1.766615,0.868667,0.001955
9,9,1.495578,0.866133,0.002018


In [201]:
naive_bayes.fit(X_train_vec, y_train)

RandomizedSearchCV(estimator=MultinomialNB(), n_jobs=8,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000110A2257190>})

In [209]:
pd.DataFrame(naive_bayes.cv_results_)[["rank_test_score", "param_alpha", "mean_test_score", "std_test_score"]]


,rank_test_score,param_alpha,mean_test_score,std_test_score
0,4,0.980259,0.787467,0.003947
1,2,0.58934,0.796867,0.004188
2,10,1.462265,0.778533,0.004884
3,7,1.089829,0.785267,0.004786
4,6,1.053396,0.786267,0.004419
5,9,1.338636,0.780867,0.004425
6,3,0.922502,0.788800,0.003500
7,1,0.101918,0.829400,0.005495
8,8,1.307344,0.781467,0.004655
9,5,1.018417,0.786467,0.004485


In [202]:
linear_svc.fit(X_train_vec, y_train)

RandomizedSearchCV(estimator=LinearSVC(), n_jobs=8,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000110A222B580>})

In [210]:
pd.DataFrame(linear_svc.cv_results_)[["rank_test_score", "param_C", "mean_test_score", "std_test_score"]]


,rank_test_score,param_C,mean_test_score,std_test_score
0,10,0.188321,0.869667,0.001776
1,1,1.606995,0.880400,0.004374
2,5,3.72581,0.879733,0.004616
3,7,0.655047,0.878800,0.002911
4,6,2.101763,0.879667,0.004467
5,4,2.077448,0.879733,0.004635
6,8,0.624893,0.878467,0.002933
7,2,1.685589,0.880200,0.004235
8,3,3.40858,0.879800,0.004740
9,9,0.35024,0.875200,0.002418


In [216]:
def print_metrics(y_pred, y_test):
    print(f'Precision: {precision_score(y_pred, y_test, average="weighted"):.4f}')
    print(f'Recall: {recall_score(y_pred, y_test, average="weighted"):.4f}')
    print(f'F1 score: {f1_score(y_pred, y_test, average="weighted"):.4f}')
    print(f'Accuracy: {accuracy_score(y_pred, y_test):.4f}')

In [217]:
for model in [log_reg, naive_bayes, linear_svc]:
    print(f'{model.estimator.__class__.__name__}:')
    y_pred = model.best_estimator_.predict(X_test_vec)
    print_metrics(y_pred, y_test)
    print()

LogisticRegression:
Precision: 0.8918
Recall: 0.8877
F1 score: 0.8888
Accuracy: 0.8877

MultinomialNB:
Precision: 0.8787
Recall: 0.8407
F1 score: 0.8548
Accuracy: 0.8407

LinearSVC:
Precision: 0.8918
Recall: 0.8890
F1 score: 0.8899
Accuracy: 0.8890

